# Concurrency Patterns - High-Level Async Coordination

Concurrency patterns provide battle-tested primitives for common async coordination scenarios:

**Core Patterns:**
- **gather**: Run multiple awaitables concurrently, collect all results
- **race**: Return first completion, cancel the rest
- **bounded_map**: Apply async function to items with concurrency limit
- **CompletionStream**: Process results as they complete (streaming)
- **retry**: Exponential backoff retry with deadline awareness

**Key Features:**
- Structured concurrency (proper cleanup on cancellation)
- Exception handling with `return_exceptions` mode
- Deadline-aware operations
- Rate limiting via `CapacityLimiter`

In [1]:
from lionherd_core.libs.concurrency import (
    CompletionStream,
    bounded_map,
    gather,
    race,
    retry,
    sleep,
)

## 1. gather() - Concurrent Execution with Result Collection

`gather()` runs multiple awaitables concurrently and collects all results. Similar to `asyncio.gather()` but with structured concurrency.

In [2]:
# Basic gather - all tasks succeed
async def fetch_data(id: int, delay: float) -> dict:
    await sleep(delay)
    return {"id": id, "data": f"result_{id}"}


# Run 3 tasks concurrently
results = await gather(
    fetch_data(1, 0.1),
    fetch_data(2, 0.05),
    fetch_data(3, 0.15),
)

print(f"Completed {len(results)} tasks:")
for r in results:
    print(f"  {r}")

Completed 3 tasks:
  {'id': 1, 'data': 'result_1'}
  {'id': 2, 'data': 'result_2'}
  {'id': 3, 'data': 'result_3'}


### Exception Handling

By default, gather raises on first error. Use `return_exceptions=True` to collect both successes and failures.

In [3]:
# Task that fails
async def maybe_fail(id: int, should_fail: bool) -> str:
    await sleep(0.01)
    if should_fail:
        raise ValueError(f"Task {id} failed")
    return f"Success {id}"


# Collect exceptions instead of raising
results = await gather(
    maybe_fail(1, False),
    maybe_fail(2, True),  # This fails
    maybe_fail(3, False),
    return_exceptions=True,
)

print("Results with exceptions:")
for i, r in enumerate(results):
    if isinstance(r, Exception):
        print(f"  Task {i}: FAILED - {r}")
    else:
        print(f"  Task {i}: {r}")

Results with exceptions:
  Task 0: Success 1
  Task 1: FAILED - Task 2 failed
  Task 2: Success 3


In [4]:
# Without return_exceptions, first error propagates
try:
    results = await gather(
        maybe_fail(1, False),
        maybe_fail(2, True),  # This will raise
        maybe_fail(3, False),
    )
except ExceptionGroup as eg:
    print(f"ExceptionGroup raised with {len(eg.exceptions)} exception(s):")
    for exc in eg.exceptions:
        print(f"  - {exc}")

ExceptionGroup raised with 1 exception(s):
  - Task 2 failed


## 2. race() - First Wins

`race()` returns the result of the first awaitable to complete, canceling all others.

In [5]:
# Different speed tasks
async def slow_api() -> str:
    await sleep(0.5)
    return "slow_result"


async def fast_api() -> str:
    await sleep(0.1)
    return "fast_result"


async def medium_api() -> str:
    await sleep(0.3)
    return "medium_result"


# Race them - fast_api wins
winner = await race(slow_api(), fast_api(), medium_api())
print(f"Winner: {winner}")

Winner: fast_result


In [6]:
# Race with timeout - useful for fallback patterns
async def primary_service() -> str:
    await sleep(1.0)  # Too slow
    return "primary"


async def timeout_fallback() -> str:
    await sleep(0.2)  # Timeout threshold
    return "timeout_exceeded"


result = await race(primary_service(), timeout_fallback())
print(f"Result: {result}")  # Falls back due to timeout

Result: timeout_exceeded


## 3. bounded_map() - Controlled Parallelism

`bounded_map()` applies an async function to many items with a concurrency limit. Essential for rate-limited APIs.

In [7]:
# Simulate API calls with rate limiting
call_count = 0
max_concurrent = 0
active_calls = 0


async def rate_limited_api(item: int) -> dict:
    global call_count, max_concurrent, active_calls

    call_count += 1
    active_calls += 1
    max_concurrent = max(max_concurrent, active_calls)

    await sleep(0.1)  # Simulate API call

    active_calls -= 1
    return {"item": item, "processed": True}


# Process 10 items with limit of 3 concurrent
items = range(10)
results = await bounded_map(
    rate_limited_api,
    items,
    limit=3,
)

print(f"Processed {len(results)} items")
print(f"Total calls: {call_count}")
print(f"Max concurrent: {max_concurrent} (limit was 3)")
print(f"Sample results: {results[:3]}")

Processed 10 items
Total calls: 10
Max concurrent: 3 (limit was 3)
Sample results: [{'item': 0, 'processed': True}, {'item': 1, 'processed': True}, {'item': 2, 'processed': True}]


### Exception Handling in bounded_map

In [8]:
# Mix of successes and failures
async def process_item(n: int) -> int:
    await sleep(0.01)
    if n % 3 == 0:
        raise ValueError(f"Item {n} is divisible by 3")
    return n * 2


# Collect exceptions
results = await bounded_map(
    process_item,
    range(10),
    limit=4,
    return_exceptions=True,
)

print("Results:")
for i, r in enumerate(results):
    if isinstance(r, Exception):
        print(f"  Item {i}: FAILED - {r}")
    else:
        print(f"  Item {i}: {r}")

Results:
  Item 0: FAILED - Item 0 is divisible by 3
  Item 1: 2
  Item 2: 4
  Item 3: FAILED - Item 3 is divisible by 3
  Item 4: 8
  Item 5: 10
  Item 6: FAILED - Item 6 is divisible by 3
  Item 7: 14
  Item 8: 16
  Item 9: FAILED - Item 9 is divisible by 3


## 4. CompletionStream - Process Results as They Arrive

`CompletionStream` yields results as tasks complete (not in submission order). Useful for UI updates or early processing.

In [9]:
# Tasks with varying completion times
async def task_with_delay(id: int, delay: float) -> dict:
    await sleep(delay)
    return {"id": id, "delay": delay}


tasks = [
    task_with_delay(1, 0.3),
    task_with_delay(2, 0.1),  # Completes first
    task_with_delay(3, 0.5),
    task_with_delay(4, 0.2),  # Completes second
]

print("Processing as tasks complete:")
async with CompletionStream(tasks) as stream:
    async for idx, result in stream:
        print(f"  Task {idx} completed: {result}")

Processing as tasks complete:
  Task 1 completed: {'id': 2, 'delay': 0.1}
  Task 3 completed: {'id': 4, 'delay': 0.2}
  Task 0 completed: {'id': 1, 'delay': 0.3}
  Task 2 completed: {'id': 3, 'delay': 0.5}


### Limited Concurrency in CompletionStream

In [10]:
# Process many tasks with concurrency limit
async def work_item(n: int) -> str:
    await sleep(0.05)
    return f"Processed_{n}"


tasks = [work_item(i) for i in range(20)]

completed = []
async with CompletionStream(tasks, limit=5) as stream:
    async for idx, result in stream:
        completed.append((idx, result))

print(f"Completed {len(completed)} tasks with limit=5")
print(f"First 5 completions: {completed[:5]}")

Completed 20 tasks with limit=5
First 5 completions: [(0, 'Processed_0'), (1, 'Processed_1'), (2, 'Processed_2'), (3, 'Processed_3'), (4, 'Processed_4')]


### Early Exit from CompletionStream

In [11]:
# Stop processing after finding target
async def search_task(id: int) -> dict:
    await sleep(0.1)
    return {"id": id, "found": id == 5}


tasks = [search_task(i) for i in range(20)]

print("Searching for target (id=5):")
async with CompletionStream(tasks) as stream:
    async for idx, result in stream:
        print(f"  Checked task {idx}: {result}")
        if result["found"]:
            print("  ✓ Found target! Canceling remaining tasks.")
            break  # Remaining tasks automatically canceled on exit

Searching for target (id=5):
  Checked task 0: {'id': 0, 'found': False}
  Checked task 1: {'id': 1, 'found': False}
  Checked task 2: {'id': 2, 'found': False}
  Checked task 3: {'id': 3, 'found': False}
  Checked task 4: {'id': 4, 'found': False}
  Checked task 5: {'id': 5, 'found': True}
  ✓ Found target! Canceling remaining tasks.


## 5. retry() - Exponential Backoff with Deadline Awareness

`retry()` implements exponential backoff retry logic with deadline awareness and jitter.

In [12]:
# Flaky operation that succeeds on 3rd attempt
attempt_count = 0


async def flaky_operation() -> str:
    global attempt_count
    attempt_count += 1

    if attempt_count < 3:
        raise ConnectionError(f"Attempt {attempt_count} failed")

    return "Success!"


# Reset counter
attempt_count = 0

# Retry with default settings (3 attempts, exponential backoff)
result = await retry(flaky_operation)
print(f"Result: {result}")
print(f"Succeeded after {attempt_count} attempts")

Result: Success!
Succeeded after 3 attempts


### Custom Retry Configuration

In [13]:
# Configure retry behavior
attempt_count = 0
delays = []


async def unstable_api() -> str:
    global attempt_count
    attempt_count += 1

    if attempt_count < 4:
        raise TimeoutError(f"Timeout on attempt {attempt_count}")

    return "API response"


# Reset
attempt_count = 0

# Custom retry parameters
result = await retry(
    unstable_api,
    attempts=5,  # Try up to 5 times
    base_delay=0.05,  # Start with 50ms delay
    max_delay=0.5,  # Cap at 500ms
    jitter=0.2,  # 20% jitter
    retry_on=(TimeoutError,),  # Only retry on TimeoutError
)

print(f"Result: {result}")
print(f"Total attempts: {attempt_count}")

Result: API response
Total attempts: 4


### Retry Exhaustion

In [14]:
# Operation that always fails
async def always_fails() -> str:
    raise RuntimeError("Permanent failure")


# Retry will exhaust and raise original exception
try:
    result = await retry(
        always_fails,
        attempts=3,
        base_delay=0.01,
    )
except RuntimeError as e:
    print(f"✓ Retry exhausted, raised: {e}")

✓ Retry exhausted, raised: Permanent failure


### Selective Retry

In [15]:
# Only retry specific exceptions
async def mixed_errors(attempt: list[int]) -> str:
    attempt[0] += 1

    if attempt[0] == 1:
        raise ConnectionError("Network flake")  # Will retry
    elif attempt[0] == 2:
        raise ValueError("Bad input")  # Won't retry - propagates immediately

    return "Success"


# Reset attempt counter
attempt = [0]

try:
    result = await retry(
        lambda: mixed_errors(attempt),
        attempts=5,
        base_delay=0.01,
        retry_on=(ConnectionError, TimeoutError),  # Only retry these
    )
except ValueError as e:
    print(f"✓ ValueError not retried, raised immediately: {e}")
    print(f"  Attempts made: {attempt[0]}")

✓ ValueError not retried, raised immediately: Bad input
  Attempts made: 2


## 6. Practical Patterns

Combining multiple patterns for real-world scenarios.

### Pattern: Parallel API Calls with Retry

In [16]:
# Fetch from multiple services with retry
services_attempted = {}


async def fetch_from_service(service: str) -> dict:
    # Track attempts
    services_attempted[service] = services_attempted.get(service, 0) + 1

    # Simulate flakiness
    if services_attempted[service] < 2:
        raise ConnectionError(f"{service} connection failed")

    await sleep(0.05)
    return {"service": service, "data": f"data_from_{service}"}


# Clear tracking
services_attempted = {}

# Gather with retry
services = ["api_1", "api_2", "api_3"]
results = await gather(
    *[retry(lambda s=svc: fetch_from_service(s), attempts=3, base_delay=0.01) for svc in services],
    return_exceptions=True,
)

print("Service results:")
for i, r in enumerate(results):
    service = services[i]
    attempts = services_attempted[service]
    if isinstance(r, Exception):
        print(f"  {service}: FAILED after {attempts} attempts - {r}")
    else:
        print(f"  {service}: {r} (succeeded on attempt {attempts})")

Service results:
  api_1: {'service': 'api_1', 'data': 'data_from_api_1'} (succeeded on attempt 2)
  api_2: {'service': 'api_2', 'data': 'data_from_api_2'} (succeeded on attempt 2)
  api_3: {'service': 'api_3', 'data': 'data_from_api_3'} (succeeded on attempt 2)


### Pattern: Rate-Limited Batch Processing

In [17]:
# Process items with rate limit and retry
processed = []


async def process_with_retry(item: int) -> dict:
    # Retry wrapper for flaky processing
    async def process() -> dict:
        await sleep(0.02)
        # Simulate occasional failures
        if item % 7 == 0 and item not in processed:
            raise ConnectionError(f"Flake on item {item}")
        processed.append(item)
        return {"item": item, "status": "processed"}

    return await retry(process, attempts=3, base_delay=0.01)


# Clear
processed = []

# Bounded map with retry
items = range(20)
results = await bounded_map(
    process_with_retry,
    items,
    limit=5,  # Max 5 concurrent
    return_exceptions=True,
)

successes = sum(1 for r in results if not isinstance(r, Exception))
failures = sum(1 for r in results if isinstance(r, Exception))

print(f"Processed {len(items)} items with limit=5:")
print(f"  Successes: {successes}")
print(f"  Failures: {failures}")

Processed 20 items with limit=5:
  Successes: 17
  Failures: 3


### Pattern: Redundant Requests (race + retry)

In [18]:
# Race multiple providers with retry
async def fetch_from_provider(name: str, base_delay: float) -> dict:
    await sleep(base_delay)
    return {"provider": name, "data": f"from_{name}"}


# Wrap each provider in retry
providers = [
    retry(lambda: fetch_from_provider("fast_provider", 0.1), attempts=2),
    retry(lambda: fetch_from_provider("slow_provider", 0.3), attempts=2),
    retry(lambda: fetch_from_provider("backup_provider", 0.2), attempts=2),
]

# Race them - fastest wins
result = await race(*providers)
print(f"Winner: {result['provider']}")
print(f"Data: {result['data']}")

Winner: fast_provider
Data: from_fast_provider


## Summary Checklist

**Concurrency Patterns:**
- ✅ `gather()`: Run awaitables concurrently, collect all results
- ✅ `race()`: Return first completion, cancel others
- ✅ `bounded_map()`: Apply async function with concurrency limit
- ✅ `CompletionStream`: Process results as they complete
- ✅ `retry()`: Exponential backoff with deadline awareness

**Exception Handling:**
- ✅ `return_exceptions=True` collects errors instead of raising
- ✅ ExceptionGroup for non-cancel errors in gather/bounded_map
- ✅ Selective retry with `retry_on` parameter

**Advanced Features:**
- ✅ Structured concurrency (proper cleanup on cancellation)
- ✅ Deadline awareness in retry
- ✅ Rate limiting via concurrency bounds
- ✅ Early exit from CompletionStream
- ✅ Jitter in exponential backoff

**Next Steps:**
- See `_primitives` for `CapacityLimiter` and other low-level primitives
- See `_cancel` for deadline and cancellation utilities
- See `_task` for task group abstractions